<a href="https://colab.research.google.com/github/GennaroBlarasin/Testing/blob/master/ConnectAEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import urllib
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Setting up database parameters

Modify only the two first lines with your credentials

In [2]:
technicalAccountId = "8EAF521762958FF50A495E8D"         #Configure this with your own ID
credential = "Conexio1@125"                             #Configure this with your own Credential

dsn_hostname = 'spnam.platform-query.adobe.io'
dsn_port = '80'
dsn_database= 'conexio3:all'
dsn_uid = '73D97EE25CCCE8260A495EBD@AdobeOrg'
dsn_pwd = technicalAccountId + ':' + credential

In [5]:
tablename_websdk = "xdm_experienceevent_conexio_dataset"
tablename_hubspot = "xdm_individual_profile_hubspot_conexio_dataset"

Write your SQL Query to get your data into """ query """

In [6]:
sql = """SELECT * FROM xdm_individual_profile_hubspot_conexio_dataset"""

It makes the connection. Original is a dataframe

In [7]:
try:
    conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
    print("Connecting to database\n  ->%s" % (conn_string))
    conn=psycopg2.connect(conn_string)
    original = pd.read_sql_query(sql, conn)
    conn = None
    print("Connected!\n")
except:
    print("Unable to connect to the database.")

Connecting to database
  ->host=spnam.platform-query.adobe.io port=80 dbname=conexio3:all user=73D97EE25CCCE8260A495EBD@AdobeOrg password=8EAF521762958FF50A495E8D:Conexio1@125
Connected!



It's a good practice make a copy in order to always have the original pull

In [8]:
data_frame = original.copy()

Let's visualize the data

In [9]:
data_frame

,homeAddress,mobilePhone,person,personalEmail,b2b,extendedWorkDetails,createdByBatchID
0,"(Boise,NULL,NULL)","(NULL, +1 403-720-4500)","(""(Ladonna,Hale)"")","(ladonna.hale@lucernefoods.com,""(lucernefoods....",(Lucerne Foods),(President),22/4/2022 13:50
1,None,None,"(""(David,""Levy (Critical Mass)"")"")","(dlevy@criticalmass.com,""(criticalmass.com)"")",None,None,25/4/2022 08:58
2,"(Tustin,NULL,NULL)","(NULL, +1 877-300-8837)","(""(Jaclyn,Cordova-Downey)"")","(jaclyn.cordova@tcl.com,""(tcl.com)"")",(TCL Communication),(Director Of Marketing Operations),22/4/2022 13:55
3,"(Huntington Beach,NULL,NULL)","(NULL, +1 877-300-8837)","(""(Ian,Nicholls)"")","(ian.nicholls@tcl.com,""(tcl.com)"")",(TCL Communication),(Director of Marketing),22/4/2022 13:55
4,"(Boise,NULL,NULL)","(NULL, +1 403-720-4500)","(""(Ladonna,Hale)"")","(ladonna.hale@lucernefoods.com,""(lucernefoods....",(Lucerne Foods),(President),22/4/2022 13:50
...,...,...,...,...,...,...,...
2173,None,None,"(""(Dave,Katz)"")","(dave.katz@annalect.com,""(annalect.com)"")",(Annalect),None,19/11/2021 09:28
2174,None,None,"(""(Lisa,Lockwood)"")","(lockwoodl@yahoo.com,""(yahoo.com)"")",None,(Head of Web and Digital Marketing),18/11/2021 17:42
2175,"(El Dorado Hills,NULL,NULL)",None,"(""(Dan,Martino)"")","(dan.martino@tadigital.com,""(tadigital.com)"")",(TA Digital),(AEM Practice Director),17/11/2021 18:30
2176,"(Chicago,NULL,NULL)","(NULL,3.127.319.628)","(""(Ali,Alkhafaji)"")","(ali.alkhafaji@tadigital.com,""(tadigital.com)"")",(TA Digital),(CTO),17/11/2021 18:30
